In [1]:
import pandas as pd
import numpy as np

I gave blastp fasta files with 20040 genes for elegans, and 20829 genes from briggsae. It compared every single elegans gene to every single briggsae gene (query=elegans, target=briggsae), and every single briggsae gene to every single elegans gene (query=briggsae, target=elegans).

My end goal is to get the e values for the 1:1 ortholog pairs that I'm using in my synteny analysis for DAGChainer. 

# Intro

In [2]:
# make dictionary to convert elegans gene to the orthologous briggsae gene name and vice versa

bed_12_columns=['chrom','chromStart','chromEnd','name','score','strand','thickStart','thickEnd','itemRgb',
        'blockCount', 'blockSizes', 'blockStart']

elegans_genes_tc=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/elegans_to_compare.bed', 
                             sep='\t', names=bed_12_columns)
briggsae_genes_tc=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/briggsae_to_compare.bed', 
                              sep='\t', names=bed_12_columns)

elegans_to_briggsae={}
for i in range(len(elegans_genes_tc)):
    elegans_to_briggsae[elegans_genes_tc['name'][i]]=briggsae_genes_tc['name'][i]
    
briggsae_to_elegans={}
for i in range(len(briggsae_genes_tc)):
    briggsae_to_elegans[briggsae_genes_tc['name'][i]]=elegans_genes_tc['name'][i]

In [3]:
# read in the blast outputs

names=['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
path_stem='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/'

# go through each briggsae gene and see the e values for a match with every elegans gene
elegans_vs_briggsae=pd.read_csv(path_stem+'elegans_target_vs_briggsae_query.txt',sep='\t',names=names)

# go through each elegans gene and see the e values for a match with every briggsae gene
briggsae_vs_elegans=pd.read_csv(path_stem+'briggsae_target_vs_elegans_query.txt',sep='\t',names=names)

In [4]:
elegans_vs_briggsae

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,WBGene00023521,WBGene00020200,53.143,175,81,1,1,174,1,175,6.970000e-56,173.0
1,WBGene00023521,WBGene00021159,38.298,47,27,1,63,109,403,447,5.000000e-01,30.0
2,WBGene00023521,WBGene00020648,35.714,42,27,0,49,90,6,47,1.200000e+00,28.9
3,WBGene00023521,WBGene00013566,35.385,65,33,3,75,138,2,58,1.300000e+00,28.5
4,WBGene00023521,WBGene00007324,24.038,104,73,2,57,157,59,159,2.400000e+00,27.3
...,...,...,...,...,...,...,...,...,...,...,...,...
987289,WBGene00304771,WBGene00005089,21.212,165,93,6,106,259,156,294,5.000000e+00,27.7
987290,WBGene00304771,WBGene00002038,25.581,86,52,1,194,267,519,604,5.200000e+00,28.1
987291,WBGene00304771,WBGene00022408,23.864,176,102,7,50,213,54,209,5.600000e+00,27.7
987292,WBGene00304771,WBGene00007633,35.000,40,25,1,15,53,88,127,7.200000e+00,26.9


In [10]:
elegans_to_briggsae['WBGene00020200']

'WBGene00023521'

In [5]:
all_b_query_genes=set()
for i in elegans_vs_briggsae['qseqid']:all_b_query_genes.add(i)
all_e_query_genes=set()
for i in briggsae_vs_elegans['qseqid']:all_e_query_genes.add(i)
    
len(all_b_query_genes),len(all_e_query_genes)

(20795, 19991)

In [6]:
elegans_all_genes=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/elegans_one_of_each_gene.bed', 
                              header=None, sep='\t')
briggsae_all_genes=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/briggsae_one_of_each_gene.bed', 
                              header=None, sep='\t')

In [7]:
briggsae_missing_genes=len(briggsae_all_genes)-len(all_b_query_genes)
elegans_missing_genes=len(elegans_all_genes)-len(all_e_query_genes)
briggsae_missing_genes, elegans_missing_genes
# number of genes that I gave to blast in the fasta file that don't appear in the final output
# I guess these must be species specific. No matches at all

(34, 49)

In [8]:
target_worm='briggsae'
query_worm='elegans'

# Make dataframes of all the blast hits between just 1:1 ortholog pairs and get the ranking of each match (ie. was the 1:1 ortholog the top hit)

In [85]:
# DON'T RUN AGAIN

# make a dataframe of all the blast hits between 1:1 ortholog pairs, and get the ranking of the match
# in some cases, there might be multiple 

if query_worm=='elegans':
    df=briggsae_vs_elegans.iloc[:,[0,1,-2]].copy()
    query_orthologs=list(elegans_genes_tc['name'])
    query_to_target=elegans_to_briggsae
    output_path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/elegans_target_briggsae_query_evals_hitnums'

if query_worm=='briggsae':
    df=elegans_vs_briggsae.iloc[:,[0,1,-2]].copy()
    query_orthologs=list(briggsae_genes_tc['name'])
    query_to_target=briggsae_to_elegans
    output_path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/briggsae_target_elegans_query_evals_hitnums'

target_gene_list=[]
query_gene_list=[]
e_values=[]
target_hit_positions=[]
    
for gene in query_orthologs:
    gene_df=df[df['qseqid']==gene].copy().reset_index(drop=True)
    gene_and_ortholog_df=gene_df[gene_df['sseqid']==query_to_target[gene]]
    for index, row in gene_and_ortholog_df.iterrows():
        target_gene_list.append(row['sseqid'])
        query_gene_list.append(row['qseqid'])
        e_values.append(row['evalue'])
        target_hit_positions.append(index)
        
output_df=pd.DataFrame({'query_gene':query_gene_list,'target_gene':target_gene_list,
                        'e_value':e_values,'target_hit_position':target_hit_positions})

output_df.to_csv(output_path,sep='\t',index=None)

In [6]:
# read in the output from the above cell

briggsae_query_orthologs=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/briggsae_target_elegans_query_evals_hitnums',
                                    sep='\t')
elegans_query_orthologs=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/elegans_target_briggsae_query_evals_hitnums',
                                   sep='\t')
briggsae_query_orthologs.columns=['query_gene','target_gene','e_value','target_hit_position']
elegans_query_orthologs.columns=['query_gene','target_gene','e_value','target_hit_position']

In [7]:
len(briggsae_query_orthologs),len(elegans_query_orthologs)

(12388, 12425)

### Comparing genes in blast output vs in blast input - what genes are missing?

In [8]:
bed_12_columns=['chrom','chromStart','chromEnd','name','score','strand','thickStart','thickEnd','itemRgb',
        'blockCount', 'blockSizes', 'blockStart']

elegans_genes_tc=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/elegans_to_compare.bed', 
                             sep='\t', names=bed_12_columns)
briggsae_genes_tc=pd.read_csv('/home/helena_hatrick/part_ii_project/canonical_genesets/briggsae_to_compare.bed', 
                              sep='\t', names=bed_12_columns)

In [47]:
# these are all the orthologs that I gave to blast

len(elegans_genes_tc),len(briggsae_genes_tc)

(11814, 11814)

In [9]:
briggsae_query_genes=set()
for i in list(briggsae_query_orthologs['query_gene']):briggsae_query_genes.add(i)
    
elegans_query_genes=set()
for i in list(elegans_query_orthologs['query_gene']):elegans_query_genes.add(i)

In [10]:
len(briggsae_query_genes),len(elegans_query_genes)

(11782, 11778)

In [11]:
b_absent_query_genes=set()

for gene in briggsae_genes_tc['name']:
    if gene not in briggsae_query_genes:
        b_absent_query_genes.add(gene)
        
e_absent_query_genes=set()

for gene in elegans_genes_tc['name']:
    if gene not in elegans_query_genes:
        e_absent_query_genes.add(gene)

In [12]:
len(b_absent_query_genes), len(e_absent_query_genes)

(32, 36)

In [13]:
for gene in b_absent_query_genes:
    if briggsae_to_elegans[gene] not in e_absent_query_genes:print('no')

no
no


In [14]:
for gene in e_absent_query_genes:
    if elegans_to_briggsae[gene] not in b_absent_query_genes:print('no')

no
no
no
no
no
no


See a lot of overlap between the genes missing from the elegans analysis and briggsae analysis. The genes don't have a hit in the other sequence.

# Get just the first row where the query gene was matched with its ortholog

The reason why there are more rows in the ortholog pair matches df above is because often the query gene gets multiple local matches with its ortholog.

In [16]:
e_q_genes=list(elegans_query_genes)
first_match_indices=[]

for i in e_q_genes:
    sub_df=elegans_query_orthologs[elegans_query_orthologs['query_gene']==i].copy().reset_index()
    first_match_indices.append(sub_df['index'][0])

e_q_first_ortholog_hit=elegans_query_orthologs.iloc[first_match_indices,:].copy()

In [17]:
b_q_genes=list(briggsae_query_genes)
first_match_indices=[]

for i in b_q_genes:
    sub_df=briggsae_query_orthologs[briggsae_query_orthologs['query_gene']==i].copy().reset_index()
    first_match_indices.append(sub_df['index'][0])

b_q_first_ortholog_hit=briggsae_query_orthologs.iloc[first_match_indices,:].copy()

# Take only those rows where the ortholog was the top hit, not a secondary hit

In [19]:
e_q_ortholog_best_hit=e_q_first_ortholog_hit[e_q_first_ortholog_hit['target_hit_position']==0].copy()
e_q_ortholog_best_hit=e_q_ortholog_best_hit.reset_index(drop=True).drop('target_hit_position',axis=1)

In [20]:
b_q_ortholog_best_hit=b_q_first_ortholog_hit[b_q_first_ortholog_hit['target_hit_position']==0].copy()
b_q_ortholog_best_hit=b_q_ortholog_best_hit.reset_index(drop=True).drop('target_hit_position',axis=1)

In [21]:
e_q_ortholog_best_hit

,query_gene,target_gene,e_value
0,WBGene00012576,WBGene00270345,0.000000e+00
1,WBGene00018027,WBGene00034838,5.080000e-22
2,WBGene00000281,WBGene00036319,0.000000e+00
3,WBGene00022051,WBGene00035537,0.000000e+00
4,WBGene00004355,WBGene00026109,0.000000e+00
...,...,...,...
11547,WBGene00019399,WBGene00033527,1.120000e-131
11548,WBGene00017513,WBGene00041256,3.470000e-94
11549,WBGene00017144,WBGene00025447,0.000000e+00
11550,WBGene00016699,WBGene00037230,0.000000e+00


In [22]:
b_q_ortholog_best_hit

,query_gene,target_gene,e_value
0,WBGene00034901,WBGene00005717,0.000000e+00
1,WBGene00029635,WBGene00008808,0.000000e+00
2,WBGene00025224,WBGene00019320,1.450000e-99
3,WBGene00270397,WBGene00013678,0.000000e+00
4,WBGene00035625,WBGene00012962,0.000000e+00
...,...,...,...
11579,WBGene00030178,WBGene00011225,0.000000e+00
11580,WBGene00039007,WBGene00016289,0.000000e+00
11581,WBGene00038091,WBGene00005027,9.970000e-129
11582,WBGene00041848,WBGene00001179,0.000000e+00


### Cases where the ortholog was a hit but not the top hit

In [31]:
len(e_q_first_ortholog_hit[e_q_first_ortholog_hit['target_hit_position']>0])

226

In [32]:
len(b_q_first_ortholog_hit[b_q_first_ortholog_hit['target_hit_position']>0])

198

Here are cases where the ortholog is a hit, but not the top hit.

In [178]:
b_q_first_ortholog_hit[b_q_first_ortholog_hit['target_hit_position']>0]

,query_gene,target_gene,e_value,target_hit_position
8314,WBGene00024098,WBGene00268213,1.670000e-19,9
11981,WBGene00041131,WBGene00008516,7.380000e-124,1
133,WBGene00029034,WBGene00001773,3.510000e-108,2
8284,WBGene00031862,WBGene00020183,1.760000e-41,2
12255,WBGene00036087,WBGene00010011,2.050000e-144,1
...,...,...,...,...
4917,WBGene00040617,WBGene00015641,9.420000e-21,4
4036,WBGene00035857,WBGene00021753,2.880000e-55,1
8796,WBGene00038593,WBGene00006231,1.880000e-154,2
11568,WBGene00025094,WBGene00007648,1.940000e-18,1


For elegans as query, I have 36 cases where briggsae ortholog was not a hit at all, and 226 cases where the briggsae ortholog was not the top hit (see later)

For briggsae as query, I have 32 cases where the elegans ortholog was not a hit at all, and 198 cases where the elegans ortholog was not the top hit.

Interestingly, some of the ortholog pairs that are not a top hit aren't annotated as orthologs on the wormbase website, but are present in the ortholog file downloaded from wormmart that I used to get my orthologs. To do with looking at %id with BLAST, doesn't always tell you genuine ortholog relationships. The wormmart stuff looks at evolutionary origins, can be trusted more than my blast analysis. BUT it's too difficult to use the genes that don't come up as top hits, so I'll just ignroe them. Could be a signal of duplications or something.

So I will discard all those ortholog pairs that are not reciprocal best hits from my blast analysis.

# Get the reciprocal best hits (RBHs)

In [34]:
b_q_ortholog_best_hit 
# all the cases where the elegans (target) ortholog was the best hit for the briggsae (query) gene

,query_gene,target_gene,e_value
0,WBGene00034901,WBGene00005717,0.000000e+00
1,WBGene00029635,WBGene00008808,0.000000e+00
2,WBGene00025224,WBGene00019320,1.450000e-99
3,WBGene00270397,WBGene00013678,0.000000e+00
4,WBGene00035625,WBGene00012962,0.000000e+00
...,...,...,...
11579,WBGene00030178,WBGene00011225,0.000000e+00
11580,WBGene00039007,WBGene00016289,0.000000e+00
11581,WBGene00038091,WBGene00005027,9.970000e-129
11582,WBGene00041848,WBGene00001179,0.000000e+00


In [35]:
e_q_ortholog_best_hit
# all the cases where the briggsae (target) ortholog was the best hit for the elegans (query) gene

,query_gene,target_gene,e_value
0,WBGene00012576,WBGene00270345,0.000000e+00
1,WBGene00018027,WBGene00034838,5.080000e-22
2,WBGene00000281,WBGene00036319,0.000000e+00
3,WBGene00022051,WBGene00035537,0.000000e+00
4,WBGene00004355,WBGene00026109,0.000000e+00
...,...,...,...
11547,WBGene00019399,WBGene00033527,1.120000e-131
11548,WBGene00017513,WBGene00041256,3.470000e-94
11549,WBGene00017144,WBGene00025447,0.000000e+00
11550,WBGene00016699,WBGene00037230,0.000000e+00


### RBHs from the persepctive of elegans as target, briggsae as query

In [ ]:
# go through every row in b_q_ortholog_best_hit
# take the briggsae query gene
# take the elegans target gene
# make a df where the elegans target gene is the query gene in e_q_ortholog_best_hit
# see if the target gene is briggsae query gene from b_q_ortholog_best_hit
# if so, take the row from b_q_ortholog_best_hit

b_q_indices=[]
for index, row in b_q_ortholog_best_hit.iterrows():
    b_q_gene=row['query_gene']
    e_t_gene=row['target_gene']
    subdf=e_q_ortholog_best_hit[e_q_ortholog_best_hit['query_gene']==e_t_gene].copy().reset_index(drop=True)
    if len(subdf)>0:
        if subdf['target_gene'][0]==b_q_gene:
            b_q_indices.append(index)

In [ ]:
len(b_q_indices)

In [40]:
# rbh from the perspective of elegans as target, briggsae as query
ortholog_rbh_et_bq=b_q_ortholog_best_hit.iloc[b_q_indices,:].copy().reset_index(drop=True)
ortholog_rbh_et_bq

,query_gene,target_gene,e_value
0,WBGene00034901,WBGene00005717,0.000000e+00
1,WBGene00029635,WBGene00008808,0.000000e+00
2,WBGene00025224,WBGene00019320,1.450000e-99
3,WBGene00270397,WBGene00013678,0.000000e+00
4,WBGene00035625,WBGene00012962,0.000000e+00
...,...,...,...
11460,WBGene00030178,WBGene00011225,0.000000e+00
11461,WBGene00039007,WBGene00016289,0.000000e+00
11462,WBGene00038091,WBGene00005027,9.970000e-129
11463,WBGene00041848,WBGene00001179,0.000000e+00


In [11]:
elegans_to_briggsae['WBGene00005717']

'WBGene00034901'

In [53]:
# save it
path='/home/helena_hatrick/part_ii_project/orthologs/121_ortholog_rbhs_bq_et'
ortholog_rbh_et_bq.to_csv(path,sep='\t',header=['query','target','e_value'],index=None)
ortholog_rbh_et_bq=pd.read_csv(path,sep='\t')

In [54]:
# just to double check that the genes I'm getting are all the 1:1 orthologs

ideal_targets=[]
for gene in ortholog_rbh_et_bq['query']:ideal_targets.append(briggsae_to_elegans[gene])
    
ideal_targets[11460:11465]

['WBGene00011225',
 'WBGene00016289',
 'WBGene00005027',
 'WBGene00001179',
 'WBGene00000890']

### RBHs from the perspective of briggsae as target, elegans as query

In [ ]:
#should give the same number of indices as b_q_indices, but the actual indices will be different:

e_q_indices=[]
for index, row in e_q_ortholog_best_hit.iterrows():
    e_q_gene=row['query_gene']
    b_t_gene=row['target_gene']
    subdf=b_q_ortholog_best_hit[b_q_ortholog_best_hit['query_gene']==b_t_gene].copy().reset_index(drop=True)
    if len(subdf)>0:
        if subdf['target_gene'][0]==e_q_gene:
            e_q_indices.append(index)

In [51]:
len(e_q_indices)

11465

In [45]:
# rbh from the perspective of briggsae as target, elegans as query
ortholog_rbh_bt_eq=e_q_ortholog_best_hit.iloc[e_q_indices,:].copy().reset_index(drop=True)
ortholog_rbh_bt_eq

,query_gene,target_gene,e_value
0,WBGene00012576,WBGene00270345,0.000000e+00
1,WBGene00018027,WBGene00034838,5.080000e-22
2,WBGene00000281,WBGene00036319,0.000000e+00
3,WBGene00022051,WBGene00035537,0.000000e+00
4,WBGene00004355,WBGene00026109,0.000000e+00
...,...,...,...
11460,WBGene00019399,WBGene00033527,1.120000e-131
11461,WBGene00017513,WBGene00041256,3.470000e-94
11462,WBGene00017144,WBGene00025447,0.000000e+00
11463,WBGene00016699,WBGene00037230,0.000000e+00


In [56]:
# save it
path='/home/helena_hatrick/part_ii_project/orthologs/121_ortholog_rbhs_eq_bt'
ortholog_rbh_bt_eq.to_csv(path,sep='\t',header=['query','target','e_value'],index=None)
ortholog_rbh_bt_eq=pd.read_csv(path,sep='\t')

In [58]:
# just to double check that the genes I'm getting are all the 1:1 orthologs

ideal_targets=[]
for gene in ortholog_rbh_bt_eq['query']:ideal_targets.append(elegans_to_briggsae[gene])
    
ideal_targets[11460:11465]

['WBGene00033527',
 'WBGene00041256',
 'WBGene00025447',
 'WBGene00037230',
 'WBGene00028195']

# That's the final product from this .ipynb

The rest is just fiddling around

# Make a dataframe of all the top hits from the blastp output

In [86]:
# make a dataframe of all the highest hits from the blastp output

query_gene_list=[]
target_gene_list=[]
e_values=[]

if target_worm=='elegans':
    df=elegans_vs_briggsae.iloc[:,[0,1,-2]].copy()
    output_path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/elegans_vs_briggsae_top_hits'

if target_worm=='briggsae':
    df=briggsae_vs_elegans.iloc[:,[0,1,-2]].copy()
    output_path='/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/briggsae_vs_elegans_top_hits'

blast_query_genes=set()
for i in list(df['qseqid']):blast_query_genes.add(i)
    
for gene in list(blast_query_genes):
    gene_df=df[df['qseqid']==gene].copy().reset_index(drop=True)
    query_gene_list.append(gene_df['qseqid'][0])
    target_gene_list.append(gene_df['sseqid'][0])
    e_values.append(gene_df['evalue'][0])
        
top_hits_df=pd.DataFrame({'query_gene':query_gene_list,'target_gene':target_gene_list,'e_value':e_values})
top_hits_df.to_csv(output_path,sep='\t',index=None)

In [77]:
briggsae_query_top_hits=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/elegans_vs_briggsae_top_hits',
           sep='\t')
elegans_query_top_hits=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/briggsae_vs_elegans_top_hits',
           sep='\t')

# How often is the ortholog pair not the top match

In [78]:
briggsae_query_top_hits

,target_gene,query_gene,e_value
0,WBGene00008195,WBGene00023879,1.030000e-73
1,WBGene00019139,WBGene00036110,2.070000e-60
2,WBGene00016203,WBGene00033258,0.000000e+00
3,WBGene00018474,WBGene00033948,0.000000e+00
4,WBGene00015097,WBGene00025838,4.590000e-130
...,...,...,...
20790,WBGene00016178,WBGene00024828,2.000000e-03
20791,WBGene00045060,WBGene00035412,1.060000e-78
20792,WBGene00000546,WBGene00087933,3.400000e+00
20793,WBGene00018481,WBGene00024739,1.110000e-42


In [82]:
briggsae_to_elegans['WBGene00033258']

'WBGene00016203'

In [84]:
# get the rows from the top_hits_df which have a pair of orthologs

briggsae_ortholog_indices=[] # the rows containing a briggsae gene with a 1;1 ortholog as query
ortholog_pair_indices=[]
briggsae_query_ortholog_without_pair=[]

for index, row in briggsae_query_top_hits.iterrows():
    if row['query_gene'] in list(briggsae_to_elegans.keys()):
        briggsae_ortholog_indices.append(index)
        if briggsae_to_elegans[row['query_gene']]==row['target_gene']:ortholog_pair_indices.append(index)
        else: briggsae_query_ortholog_without_pair.append(index)

In [85]:
len(briggsae_ortholog_indices)

11814

In [86]:
len(ortholog_pair_indices)

11584

In [89]:
len(briggsae_query_ortholog_without_pair) 
#230 cases where there is a briggsae ortholog and it is not matched to its elegans 1:1 ortholog

230

In [56]:
len(blast_query_genes)

20795

In [82]:
elegans_vs_briggsae_evals_hitnums=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/elegans_vs_briggsae_evals_hitnums',sep='\t')
#briggsae_vs_elegans_evals_hitnums=pd.read_csv('/home/helena_hatrick/part_ii_project/protein_sets/run_blastp/briggsae_vs_elegans_evals_hitnums',sep='\t')

In [83]:
len(elegans_vs_briggsae_evals_hitnums)
#len(briggsae_vs_elegans_evals_hitnums)

12388